# Transform AFIDS between macaque templates using RheMAP transforms
**Authors:** Nikoloz Sirmpilatze (NSirmpilatze@dpz.eu)

Five volumetric macaque MRI templates are used:
* NMT v1.3
* D99
* INIA19
* MNI macaque
* Yerkes 19

The between-template transforms come from RheMAP. There are 2 kinds of transforms for each template pair:
* Linear transforms: affine, 12 degrees of freedom
* Final transforms: composite warps combining the above affine with a non-linear warp computed using ANTs SyN

## Import libraries

In [1]:
import os
from itertools import combinations
# Import function form local py file
from utils import transform_fcsv

## Define paths 

In [2]:
os.chdir(os.getcwd())

# prefixes (RheMAP and AFIDs use different prefixes for the templates)
rhemap_prefixes = ['NMTv1.3', 'D99', 'INIA', 'MNI', 'YRK']
afids_prefixes = ['nmtv1.3', 'd99', 'inia19', 'macaqueMNI', 'yerkes19']
# make dictionary matching RheMAP prefixes to AFIDs prefixes
prefix_dict = {key: value for (key, value) in zip(rhemap_prefixes, afids_prefixes)}

# Warps (Replace with your local RheMAP warp paths)
warps = '/home/niko/MRI/RheMAP/warps/'
warps_linear = os.path.join(warps, 'linear/') # linear: Affine only
warps_final = os.path.join(warps, 'final/') # final: composite Affine + SyN

# fiducials (stored within the repository)
repo_path = os.path.dirname(os.getcwd()) + '/'
# mean fiducials for each template
fid_inputs = os.path.join(repo_path, 'data/PHASE1_output_afid_postQC/')
# folder in which tranformed fiducials will be saved
fid_outputs = os.path.join(repo_path, 'data/PHASE3_RheMAP_transformed_NEW/')

## Apply transforms to points
**Important note**: ANTs transforms on points work in the opposite direction (as compared to image transforms).

For example, let's imagine we have to go from space A to space B either using a linear transform (.mat) or a non-linear warp(.nii.gz)
* Linear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B.mat`
* Linear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B -t [A-to-B.mat, 1]` (inverting the forward transform)
* Nonlinear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B_1Warp.nii.gz` (for RheMAP we would use the `A-to-B_CompositeWarp.nii.gz`)
* Nonlinear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B A-to-B_1InverseWarp.nii.gz` (for RheMAP we would use the `B-to-A_CompositeWarp.nii.gz`)

In [5]:
# Iterate over unique template pairs
for rheA, rheB in combinations(rhemap_prefixes, 2):

    # prefixes
    afdA, afdB = prefix_dict[rheA], prefix_dict[rheB]
    print(f'Transforming fiducials between {afdA} and {afdB}...')

    # input fiducials
    fcsv_A = os.path.join(fid_inputs + f'{afdA}_MEAN_QC.fcsv')
    fcsv_B = os.path.join(fid_inputs + f'{afdB}_MEAN_QC.fcsv')

    # output fiducials
    fcsv_AinB_linear = os.path.join(fid_outputs, f'{afdA}_MEAN_QC_in_{afdB}_linear.fcsv')
    fcsv_AinB_final = os.path.join(fid_outputs, f'{afdA}_MEAN_QC_in_{afdB}_final.fcsv')
    fcsv_BinA_linear = os.path.join(fid_outputs, f'{afdB}_MEAN_QC_in_{afdA}_linear.fcsv')
    fcsv_BinA_final = os.path.join(fid_outputs, f'{afdB}_MEAN_QC_in_{afdA}_final.fcsv')

    # transforms
    AtoB_linear = os.path.join(warps_linear, f'{rheA}_to_{rheB}_affine_0GenericAffine.mat')
    AtoB_final = os.path.join(warps_final, f'{rheA}_to_{rheB}_CompositeWarp.nii.gz')
    BtoA_final = os.path.join(warps_final, f'{rheB}_to_{rheA}_CompositeWarp.nii.gz')

    # apply A to B transforms
    transform_fcsv(fcsv_A, fcsv_AinB_linear, AtoB_linear, invert=1)
    print(f'{afdA} fiducials linearly transformed to {afdB} space')
    transform_fcsv(fcsv_A, fcsv_AinB_final, BtoA_final)
    print(f'{afdA} fiducials nonlinearly transformed to {afdB} space')

    # apply B to A transforms
    transform_fcsv(fcsv_B, fcsv_BinA_linear, AtoB_linear, invert=0)
    print(f'{afdB} fiducials linearly transformed to {afdA} space')
    transform_fcsv(fcsv_B, fcsv_BinA_final, AtoB_final)
    print(f'{afdB} fiducials nonlinearly transformed to {afdA} space\n')

Transforming fiducials between nmtv1.3 and d99...
nmtv1.3 fiducials linearly transformed to d99 space
nmtv1.3 fiducials nonlinearly transformed to d99 space
d99 fiducials linearly transformed to nmtv1.3 space
d99 fiducials nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and inia19...
nmtv1.3 fiducials linearly transformed to inia19 space
nmtv1.3 fiducials nonlinearly transformed to inia19 space
inia19 fiducials linearly transformed to nmtv1.3 space
inia19 fiducials nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and macaqueMNI...
nmtv1.3 fiducials linearly transformed to macaqueMNI space
nmtv1.3 fiducials nonlinearly transformed to macaqueMNI space
macaqueMNI fiducials linearly transformed to nmtv1.3 space
macaqueMNI fiducials nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and yerkes19...
nmtv1.3 fiducials linearly transformed to yerkes19 space
nmtv1.3 fiducials nonlinearly transformed t